## CONECTANDO API

Vamos utilizar uma api para extrir os dados de cotação de cambio historico de 2 meses. A API sua da foi APILayer
 (https://apilayer.com/marketplace/exchangerates_data-api#documentation-tab).
 Voce pode fazer ate 200 requisições free por mes. 

Para o codigo abaixo faça cadastro na API e altere a APIKEY para a sua.

In [0]:
dbutils.widgets.text("data_execucao", "")
data_execucao = dbutils.widgets.get("data_execucao")

In [0]:
import requests
from pyspark.sql.functions import lit

In [0]:
def extraindo_dados(date, base="BRL"):
    url = f"https://api.apilayer.com/exchangerates_data/{date}?&base={base}"
    headers = {
        "apikey": "sua chave"
    }

    parametros = {"base":base}
    response = requests.get(url, headers=headers, params=parametros)

    if response.status_code != 200:
        raise Exception("Não consegui extrair os dados")
    return response.json()

In [0]:
def dados_para_dataframe(dado_json): 
    dados_tupla = [(moeda, float(taxa)) for moeda, taxa in dado_json["rates"].items()]
    return dados_tupla

In [0]:
def salvar_arquivo_parquet(conversao_extraidas):


    ano, mes, dia = conversao_extraidas["date"].split("-")
    caminho_completo = f"dbfs:/databricks-results/bronze/{ano}/{mes}/{dia}"
    response = dados_para_dataframe(conversao_extraidas)
    df_conversoes = spark.createDataFrame(response, schema=['moeda', 'taxa'])
    df_conversoes = df_conversoes.withColumn("data", lit(f"{ano}/{mes}/{dia}"))
    df_conversoes.write.mode("overwrite").format("parquet").save(caminho_completo)
    
    print(f"Arquivo salvo com sucesso em {caminho_completo}")

In [0]:
cotacoes = extraindo_dados(data_execucao)

In [0]:
salvar_arquivo_parquet(cotacoes)